# Automatic Speech Recognition

In [ ]:
# libraries to be installed

# !pip install transformers
# !pip install -U datasets
# !pip install soundfile
# !pip install librosa
# !pip install gradio

In [ ]:
from transformers.utils import logging
logging.set_verbosity_error()

## Data Preparation

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("librispeech_asr",
                       split="train.clean.100",
                       streaming=True,
                       trust_remote_code=True)
# to load in interable dataset, streaming=True

In [ ]:
example = next(iter(dataset))

In [ ]:
dataset_head = dataset.take(5)
list(dataset_head)

In [ ]:
list(dataset_head)[2]

In [ ]:
example

In [ ]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(example["audio"]["array"],
             rate=example["audio"]["sampling_rate"])

## Building a data pipeline

In [ ]:
from transformers import pipeline

In [ ]:
# automatic speech recognition pipeline
asr = pipeline(task="automatic-speech-recognition",
               model="./models/distil-whisper/distil-small.en")

In [ ]:
asr.feature_extractor.sampling_rate

In [ ]:
example['audio']['sampling_rate']

In [ ]:
asr(example["audio"]["array"])

In [ ]:
example["text"]

## Building a Gradio app

In [ ]:
import os
import gradio as gr

In [ ]:
demo = gr.Blocks()

In [ ]:
def transcribe_speech(filepath):
    if filepath is None:
        gr.Warning("No audio found, please retry.")
        return ""
    output = asr(filepath)
    return output["text"]

In [ ]:
mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="microphone",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never")

In [ ]:
file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="upload",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never",
)

In [ ]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe,
         file_transcribe],
        ["Transcribe Microphone",
         "Transcribe Audio File"],
    )

demo.launch(share=True, 
            server_port=int(os.environ['PORT1']))

In [ ]:
demo.close()

## For longer audio files

In [ ]:
import soundfile as sf
import io

In [ ]:
audio,sampling_rate=sf.read('narration_example.wav')

In [ ]:
sampling_rate

In [ ]:
asr.feature_extractor.sampling_rate

In [ ]:
# asr(audio)

# could throw error due to multichannel audio input

### Convert the audio from stereo to mono (Using librosa)

In [ ]:
audio.shape

In [ ]:
import numpy as np

audio_transposed = np.transpose(audio)

In [ ]:
audio_transposed.shape

In [ ]:
import librosa

In [ ]:
audio_mono = librosa.to_mono(audio_transposed)

In [ ]:
IPythonAudio(audio_mono,rate=sampling_rate)

In [ ]:
# asr(audio_mono)

# the above may give wrong output due to difference in sampling rate

In [ ]:
sampling_rate

In [ ]:
asr.feature_extractor.sampling_rate

In [ ]:
audio_16KHz = librosa.resample(audio_mono,
                               orig_sr=sampling_rate,
                               target_sr=16000)

In [ ]:
asr(
    audio_16KHz,
    chunk_length_s=30, # 30 seconds
    batch_size=4,
    return_timestamps=True,
)["chunks"]

# processing in batches
# renning each batch simultaneouly means theat number of models together
# theis will also use some overlapping and then connect the final result
# the results will be chunks of audio with timestanp

### Building Gradio Interface

In [ ]:
import gradio as gr
demo = gr.Blocks()

In [ ]:
def transcribe_long_form(filepath):
    if filepath is None:
        gr.Warning("No audio found, please retry.")
        return ""
    output = asr(
      filepath,
      max_new_tokens=256,
      chunk_length_s=30,
      batch_size=8,
    )
    return output["text"]

In [ ]:
mic_transcribe = gr.Interface(
    fn=transcribe_long_form,
    inputs=gr.Audio(sources="microphone",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never")

file_transcribe = gr.Interface(
    fn=transcribe_long_form,
    inputs=gr.Audio(sources="upload",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never",
)

In [ ]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe,
         file_transcribe],
        ["Transcribe Microphone",
         "Transcribe Audio File"],
    )
demo.launch(share=True, 
            server_port=int(os.environ['PORT1']))

In [ ]:
demo.close()